In [3]:
import math
import pyscreenshot
import time
import pyautogui
import cv2
import numpy as np
import random
import copy
import ast
import os.path
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from IPython.display import clear_output
import ast
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

import keras
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential, Model, load_model
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Input

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups

import numpy as np
import os

Using TensorFlow backend.
d:\users\kaarel\anaconda3\envs\tetrisbot\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
d:\users\kaarel\anaconda3\envs\tetrisbot\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
d:\users\kaarel\anaconda3\envs\tetrisbot\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
d:\users\kaarel\anaconda3\envs\t

In [5]:
#Let's define some functions to play the game and save the data

#This finds the position of the game on the current screen
def autoCalibrate():
    topLeft  = pyautogui.locateOnScreen('hold.png')
    bottomRight = pyautogui.locateOnScreen('next.png')
    x_pad = topLeft[0] - 12
    y_pad = topLeft[1] - 47
    x_width = bottomRight[0] + 176
    y_length = bottomRight[1] + 550
    return (x_pad, y_pad, x_width, y_length)

#This captures the screen and converts the gamefield into a matrix format
def getAllBlocks():
    # Define the matrix for all the blocks
    x_size = 10
    y_size = 20
    gameMatrix = [[0 for x in range(x_size)] for y in range(y_size)]
    # Define the color that can't be a block
    black = (0,0,0)
    # Read the frame and translate it into HSV colorspace
    frame = cv2.imread("game.png")
    frame = frame[36:560, 214:482]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Fill the matrix with ones and zeros, so all the blocks would be displayed correctly
    for y in range(y_size):
        for x in range(x_size):
            x1 = 13 + (26*x)
            y1 = (560-36-13) - (26*y)
            r, g, b = frame[y1, x1]
            if (r,g,b) == black:
                gameMatrix[y][x] = 0
            else:
                gameMatrix[y][x] = 1
    return gameMatrix

#This takes an image and figures out which tetromino is displayed
#Then it returns an index for that tetromino
def getNextBlock(frame):
    # Read in all the shape files and make them into grayscale
    yellow = cv2.imread("yellow.png")
    purple = cv2.imread("purple.png")
    red = cv2.imread("red.png")
    blue = cv2.imread("blue.png")
    teal = cv2.imread("teal.png")
    green = cv2.imread("green.png")
    orange = cv2.imread("orange.png")
    yellow = cv2.cvtColor(yellow, cv2.COLOR_BGR2GRAY)
    purple = cv2.cvtColor(purple, cv2.COLOR_BGR2GRAY)
    red = cv2.cvtColor(red, cv2.COLOR_BGR2GRAY)
    blue = cv2.cvtColor(blue, cv2.COLOR_BGR2GRAY)
    teal = cv2.cvtColor(teal, cv2.COLOR_BGR2GRAY)
    green = cv2.cvtColor(green, cv2.COLOR_BGR2GRAY)
    orange = cv2.cvtColor(orange, cv2.COLOR_BGR2GRAY)
    # Match the template wih the frame
    res1 = cv2.matchTemplate(frame,yellow,cv2.TM_CCOEFF_NORMED)
    res2 = cv2.matchTemplate(frame,purple,cv2.TM_CCOEFF_NORMED)
    res3 = cv2.matchTemplate(frame,red,cv2.TM_CCOEFF_NORMED)
    res4 = cv2.matchTemplate(frame,blue,cv2.TM_CCOEFF_NORMED)
    res5 = cv2.matchTemplate(frame,teal,cv2.TM_CCOEFF_NORMED)
    res6 = cv2.matchTemplate(frame,green,cv2.TM_CCOEFF_NORMED)
    res7 = cv2.matchTemplate(frame,orange,cv2.TM_CCOEFF_NORMED)
    # Determine whether the block was detected or not
    threshold = 0.8
    match1 = False
    if np.amax(res1) > threshold:
        match1 = True
    match2 = False
    if np.amax(res2) >= threshold:
        match2 = True
    match3 = False
    if np.amax(res3) >= threshold:
        match3 = True
    match4 = False
    if np.amax(res4) >= threshold:
        match4 = True
    match5 = False
    if np.amax(res5) >= threshold:
        match5 = True
    match6 = False
    if np.amax(res6) >= threshold:
        match6 = True
    match7 = False
    if np.amax(res7) >= threshold:
        match7 = True
    # Return the block number in order to determin what block was detected
    if match1:
        return 1
    elif match2:
        return 2
    elif match3:
        return 3
    elif match4:
        return 4
    elif match5:
        return 5
    elif match6:
        return 6
    elif match7:
        return 7
    else:
        #print("getNextBlock didn't match.")
        raise ValueError("getNextBlock didn't match.")
#         print([res1, res2, res3, res4, res5, res6, res7])
#         cv2.imshow('image',frame)
#         cv2.waitKey(0)

#This starts the game by pressing on the "play" button
def startGame():
    x, y = pyautogui.locateCenterOnScreen('play.png')
    pyautogui.click(x, y)  

#This function will make a move
#x - how many squares to the right should the block be dropped? (0 to 9)
def makeMove(x, x_pad, y_pad):
    pyautogui.click(x_pad+230 + 26*x, y_pad+100)  

#This function will pause and quit the game
def endGame(x_pad, y_pad):
    pyautogui.click(x_pad+700, y_pad+540)
    time.sleep(0.1)
    pyautogui.click(x_pad+360, y_pad+400)
    time.sleep(0.1)
    pyautogui.click(x_pad+360, y_pad+300) 

#Get the height of the tower of blocks in the game matrix
def getHeight(gameMatrix):
    matrix = gameMatrix[:14]
    height = 0
    for row in range(14):
        if matrix[row] != [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]:
            height += 1
    return height



#This adds the training results to the current results in the text document
def saveTestResults(goodMoveStates, goodMoves):
    #If the file exists, add the current testing results to the text document
    if(os.path.isfile("train_data.txt")):
        file = open("train_data.txt", "r")
        dataString = file.readlines()
        file.close()
        X = ast.literal_eval(dataString[0])
        y = ast.literal_eval(dataString[1])
        X = X + goodMoveStates
        y = y + goodMoves

        text_file = open("train_data.txt", "w")
        text_file.write(str(X))
        text_file.write('\n')
        text_file.write(str(y))
        text_file.close()
    else:
        text_file = open("train_data.txt", "w")
        text_file.write(str(goodMoveStates))
        text_file.write('\n')
        text_file.write(str(goodMoves))
        text_file.close()

In [58]:
#THIS CODE BLOCK IS FOR INITIAL TRAINING
#It uses random to get its moves
#Keep making moves in a loop. 
#If the total height of the tower was reduced, save the move for training
time.sleep(5) #Wait for 5 seconds so the user can start up the game
x_pad, y_pad, x_width, y_length = autoCalibrate()
goodMoveStates = [] #unprocessed train_X
goodMoves = [] #unprocessed train_y
numberOfGames = 5000
try:
    for i in range(numberOfGames):
        clear_output()
        print("Progress: game " + str(i+1) + " out of " + str(numberOfGames))

        startGame()
        time.sleep(1) #This is so the game will have time to load
        giveUp = False #This will be used to end the loop
        lastMove = None #Data about the previous move
        lastMoveState = None #State of the game at prev. move
        lastHeight = -1 #Height of the prev. gamestate

        #Let's get the current block during the countdown
        im=pyscreenshot.grab(bbox =(x_pad,y_pad,x_width,y_length))
        im.save("game.png")

        frame = cv2.imread("game.png")
        frame1 = frame[100:172, 546:668]
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2 = frame[172:244, 546:668]
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        frame3 = frame[244:309, 546:668]
        frame3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)

        currentBlock = getNextBlock(frame1)

        time.sleep(3) #Let's wait for 3 more seconds so the countdown menu ends

        while not giveUp:
            im=pyscreenshot.grab(bbox =(x_pad,y_pad,x_width,y_length))
            im.save("game.png")

            frame = cv2.imread("game.png")
            frame1 = frame[100:172, 546:668]
            frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            frame2 = frame[172:244, 546:668]
            frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
            frame3 = frame[244:309, 546:668]
            frame3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)

            gameMatrix = getAllBlocks()
            h = getHeight(gameMatrix)
            nextBlock1 = getNextBlock(frame1)
            nextBlock2 = getNextBlock(frame2)
            nextBlock3 = getNextBlock(frame3)

            #If the tower is over 10 blocks tall, the bot ends the game
            if(h > 10):
                giveUp = True

            move = random.randint(0, 9)

            if(not giveUp):
                makeMove(move, x_pad, y_pad)

            #If the last move reduced the height of the tower,
            #it was a good move and will be saved for training
            if(h < lastHeight):
            #if(True): #For testing purposes
                goodMoveStates.append(lastMoveState)
                goodMoves.append(lastMove)

            #Save the current data for future use
            lastHeight = h
            lastMoveState = [copy.deepcopy(gameMatrix[:10]), copy.copy(currentBlock)]
            lastMove = move
            currentBlock = nextBlock1
        #close the game and start a new one
        endGame(x_pad, y_pad)
        #Every 100 games, save the results and continue
        if(str(i)[-2:] == "00"):
            saveTestResults(goodMoveStates, goodMoves)
            goodMoveStates = []
            goodMoves = []
    saveTestResults(goodMoveStates, goodMoves)
    print("data saved")
except:
    saveTestResults(goodMoveStates, goodMoves)
    print("exception encountered, data saved")

Progress: game 2252 out of 5000
exception encountered, data saved


In [6]:
#Let's define some functions to process the data from the model

#Code to read training data from the text document
def readData():

    file = open("train_data.txt", "r")
    dataString = file.readlines()
    file.close()
    X = ast.literal_eval(dataString[0])
    y = ast.literal_eval(dataString[1])

    return (X, y)

#Code for pre-processing the data
def preProcessData(X, y=[]):
    #X = np.array(X)
    X = copy.deepcopy(X)
    y = copy.deepcopy(y)
    for i in range(len(X)):
        el = X[i]
        #Let's flatten the coordinate matrix so that it is easier to process
        newMatrix = np.array(el[0]).flatten()
        #Let's convert data for the tile to an appropriate one-hot vector
        currentTile = to_categorical([el[1]-1, 6])[0]
        #Now let's add these two lists together
        newMatrix = np.array(np.append(newMatrix, currentTile))
        X[i] = newMatrix
    
    if y != []:
        y = to_categorical(y)
    return (np.array(X), np.array(y))

#Code for predicting the next move
def predict(gameMatrix, currentBlock):
    moveState = [copy.deepcopy(gameMatrix[:10]), copy.copy(currentBlock)]
    modelInput = preProcessData([moveState])[0]
    result = model.predict(modelInput)[0]
    
    return np.argmax(result)

In [7]:
input_size=107
#Let's create the neural network for the training data.
#A lot of this code is from the 10th practice lesson in the AI course.

# Ära näita hoiatusi, üksnes vigu
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

model = Sequential()
# Tuttav kõige tavalisem närvivõrgukiht
model.add(Dense(128,input_shape=(input_size,)))
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(64))
model.add(Dense(32))
# Dropout - põhimõtteliselt Dense kiht, milles osa sisendi signaal nullitakse,
#           et saavutda robustsemat mudelit.
#model.add(Dropout(0.5))
# Väljund on samal kujul, mis y: 4-kohaline vektor.
model.add(Dense(10, activation='softmax'))

model.compile('adam', 'categorical_crossentropy', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               13824     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 10)                330       
Total params: 32,810
Trainable params: 32,810
Non-trainable params: 0
_________________________________________________________________


In [8]:
#Load the data
(X, y) = readData()

#Process the data
(X, y) = preProcessData(X, y)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

#Train the model
model.fit(X_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=[X_test, y_test])

Train on 2667 samples, validate on 667 samples
Epoch 1/5
2667/2667 [==============================] - 0s - loss: 1.6876 - acc: 0.4147 - val_loss: 1.2443 - val_acc: 0.5817
Epoch 2/5
2667/2667 [==============================] - 0s - loss: 1.1215 - acc: 0.6209 - val_loss: 1.1319 - val_acc: 0.6402
Epoch 3/5
2667/2667 [==============================] - 0s - loss: 0.9705 - acc: 0.6757 - val_loss: 1.0636 - val_acc: 0.6567
Epoch 4/5
2667/2667 [==============================] - 0s - loss: 0.8933 - acc: 0.7105 - val_loss: 0.9674 - val_acc: 0.7016
Epoch 5/5
2667/2667 [==============================] - 0s - loss: 0.8779 - acc: 0.7199 - val_loss: 1.0181 - val_acc: 0.6717


In [110]:
newData = 0
try:
    #play the game using the model's predictions and makeMove()
    #Keep making moves in a loop. 
    #If the total height of the tower was reduced, save the move for training
    time.sleep(5) #Wait for 5 seconds so the user can start up the game
    x_pad, y_pad, x_width, y_length = autoCalibrate()
    goodMoveStates = [] #unprocessed train_X
    goodMoves = [] #unprocessed train_y
    numberOfGames = 2000
    for i in range(numberOfGames):
        clear_output()
        print("Progress: game " + str(i+1) + " out of " + str(numberOfGames))

        startGame()
        time.sleep(1) #This is so the game will have time to load
        giveUp = False #This will be used to end the loop
        lastMove = None #Data about the previous move
        lastMoveState = None #State of the game at prev. move
        lastHeight = -1 #Height of the prev. gamestate

        #Let's get the current block during the countdown
        im=pyscreenshot.grab(bbox =(x_pad,y_pad,x_width,y_length))
        im.save("game.png")

        frame = cv2.imread("game.png")
        frame1 = frame[100:172, 546:668]
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2 = frame[172:244, 546:668]
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        frame3 = frame[244:309, 546:668]
        frame3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)

        currentBlock = getNextBlock(frame1)

        time.sleep(3) #Let's wait for 3 more seconds so the countdown menu ends

        while not giveUp:
            im=pyscreenshot.grab(bbox =(x_pad,y_pad,x_width,y_length))
            im.save("game.png")

            frame = cv2.imread("game.png")
            frame1 = frame[100:172, 546:668]
            frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
            frame2 = frame[172:244, 546:668]
            frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
            frame3 = frame[244:309, 546:668]
            frame3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)

            gameMatrix = getAllBlocks()
            h = getHeight(gameMatrix)
            nextBlock1 = getNextBlock(frame1)
            nextBlock2 = getNextBlock(frame2)
            nextBlock3 = getNextBlock(frame3)

            #If the tower is over 10 blocks high, the bot ends the game
            if(h > 10):
                giveUp = True

            move = predict(gameMatrix, currentBlock)

            if(not giveUp):
                makeMove(move, x_pad, y_pad)

            #If the last move reduced the height of the tower,
            #it was a good move and will be saved for training
            if(h < lastHeight):
            #if(True): #For testing purposes
                goodMoveStates.append(lastMoveState)
                goodMoves.append(lastMove)
                newData += 1

            #Save the current data for future use
            lastHeight = h
            lastMoveState = [copy.deepcopy(gameMatrix[:10]), copy.copy(currentBlock)]
            lastMove = move
            currentBlock = nextBlock1
        #close the game and start a new one
        endGame(x_pad, y_pad)
        #Every 100 games, save the results and continue
        if(str(i)[-2:] == "00"):
            saveTestResults(goodMoveStates, goodMoves)
            goodMoveStates = []
            goodMoves = []
    saveTestResults(goodMoveStates, goodMoves)
    print("gathered " + str(newData) + " new datapoints!")
except:
    saveTestResults(goodMoveStates, goodMoves)
    print("gathered " + str(newData) + " new datapoints!")

Progress: game 11 out of 2000
gathered 15 new datapoints!


In [9]:
#This will save the current model to file
model.save("mudel.h5")

In [ ]:
#This will load the current model from file
modelFromDisk = load_model("mudel.h5")

In [5]:
#All code for the release version is in this cell
#The release version will load the model from memory and start playing
#This version will not record the results or re-train the model
#It will only have the necessary imports

from keras.models import load_model
import math
import pyscreenshot
import time
import pyautogui
import cv2
import numpy as np
from IPython.display import clear_output
import copy
from keras.utils import to_categorical

model = load_model("mudel.h5")


numberOfGames = 10 #How many games will be played
completedLines = 0 #Variable that stores the number of successful drops by bot
#Let's define some functions to play the game and save the data

#This finds the position of the game on the current screen
def autoCalibrate():
    topLeft  = pyautogui.locateOnScreen('hold.png')
    bottomRight = pyautogui.locateOnScreen('next.png')
    x_pad = topLeft[0] - 12
    y_pad = topLeft[1] - 47
    x_width = bottomRight[0] + 176
    y_length = bottomRight[1] + 550
    return (x_pad, y_pad, x_width, y_length)

#This captures the screen and converts the gamefield into a matrix format
def getAllBlocks():
    # Define the matrix for all the blocks
    x_size = 10
    y_size = 20
    gameMatrix = [[0 for x in range(x_size)] for y in range(y_size)]
    # Define the color that can't be a block
    black = (0,0,0)
    # Read the frame and translate it into HSV colorspace
    frame = cv2.imread("game.png")
    frame = frame[36:560, 214:482]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    # Fill the matrix with ones and zeros, so all the blocks would be displayed correctly
    for y in range(y_size):
        for x in range(x_size):
            x1 = 13 + (26*x)
            y1 = (560-36-13) - (26*y)
            r, g, b = frame[y1, x1]
            if (r,g,b) == black:
                gameMatrix[y][x] = 0
            else:
                gameMatrix[y][x] = 1
    return gameMatrix

#This takes an image and figures out which tetromino is displayed
#Then it returns an index for that tetromino
def getNextBlock(frame):
    # Read in all the shape files and make them into grayscale
    yellow = cv2.imread("yellow.png")
    purple = cv2.imread("purple.png")
    red = cv2.imread("red.png")
    blue = cv2.imread("blue.png")
    teal = cv2.imread("teal.png")
    green = cv2.imread("green.png")
    orange = cv2.imread("orange.png")
    yellow = cv2.cvtColor(yellow, cv2.COLOR_BGR2GRAY)
    purple = cv2.cvtColor(purple, cv2.COLOR_BGR2GRAY)
    red = cv2.cvtColor(red, cv2.COLOR_BGR2GRAY)
    blue = cv2.cvtColor(blue, cv2.COLOR_BGR2GRAY)
    teal = cv2.cvtColor(teal, cv2.COLOR_BGR2GRAY)
    green = cv2.cvtColor(green, cv2.COLOR_BGR2GRAY)
    orange = cv2.cvtColor(orange, cv2.COLOR_BGR2GRAY)
    # Match the template wih the frame
    res1 = cv2.matchTemplate(frame,yellow,cv2.TM_CCOEFF_NORMED)
    res2 = cv2.matchTemplate(frame,purple,cv2.TM_CCOEFF_NORMED)
    res3 = cv2.matchTemplate(frame,red,cv2.TM_CCOEFF_NORMED)
    res4 = cv2.matchTemplate(frame,blue,cv2.TM_CCOEFF_NORMED)
    res5 = cv2.matchTemplate(frame,teal,cv2.TM_CCOEFF_NORMED)
    res6 = cv2.matchTemplate(frame,green,cv2.TM_CCOEFF_NORMED)
    res7 = cv2.matchTemplate(frame,orange,cv2.TM_CCOEFF_NORMED)
    # Determine whether the block was detected or not
    threshold = 0.8
    match1 = False
    if np.amax(res1) > threshold:
        match1 = True
    match2 = False
    if np.amax(res2) >= threshold:
        match2 = True
    match3 = False
    if np.amax(res3) >= threshold:
        match3 = True
    match4 = False
    if np.amax(res4) >= threshold:
        match4 = True
    match5 = False
    if np.amax(res5) >= threshold:
        match5 = True
    match6 = False
    if np.amax(res6) >= threshold:
        match6 = True
    match7 = False
    if np.amax(res7) >= threshold:
        match7 = True
    # Return the block number in order to determin what block was detected
    if match1:
        return 1
    elif match2:
        return 2
    elif match3:
        return 3
    elif match4:
        return 4
    elif match5:
        return 5
    elif match6:
        return 6
    elif match7:
        return 7
    else:
        #print("getNextBlock didn't match.")
        raise ValueError("getNextBlock didn't match.")
#         print([res1, res2, res3, res4, res5, res6, res7])
#         cv2.imshow('image',frame)
#         cv2.waitKey(0)

#This starts the game by pressing on the "play" button
def startGame():
    x, y = pyautogui.locateCenterOnScreen('play.png')
    pyautogui.click(x, y)  

#This function will make a move
#x - how many squares to the right should the block be dropped? (0 to 9)
def makeMove(x, x_pad, y_pad):
    pyautogui.click(x_pad+230 + 26*x, y_pad+100)  

#This function will pause and quit the game
def endGame(x_pad, y_pad):
    pyautogui.click(x_pad+700, y_pad+540)
    time.sleep(0.1)
    pyautogui.click(x_pad+360, y_pad+400)
    time.sleep(0.1)
    pyautogui.click(x_pad+360, y_pad+300) 

#Get the height of the tower of blocks in the game matrix
def getHeight(gameMatrix):
    matrix = gameMatrix[:14]
    height = 0
    for row in range(14):
        if matrix[row] != [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]:
            height += 1
    return height
        

#Let's define some functions to process the data from the model

#Code to read training data from the text document
def readData():

    file = open("train_data.txt", "r")
    dataString = file.readlines()
    file.close()
    X = ast.literal_eval(dataString[0])
    y = ast.literal_eval(dataString[1])

    return (X, y)

#Code for pre-processing the data
def preProcessData(X, y=[]):
    #X = np.array(X)
    X = copy.deepcopy(X)
    y = copy.deepcopy(y)
    for i in range(len(X)):
        el = X[i]
        #Let's flatten the coordinate matrix so that it is easier to process
        newMatrix = np.array(el[0]).flatten()
        #Let's convert data for the tile to an appropriate one-hot vector
        currentTile = to_categorical([el[1]-1, 6])[0]
        #Now let's add these two lists together
        newMatrix = np.array(np.append(newMatrix, currentTile))
        X[i] = newMatrix
    
    if y != []:
        y = to_categorical(y)
    return (np.array(X), np.array(y))

#Code for predicting the next move
def predict(gameMatrix, currentBlock):
    moveState = [copy.deepcopy(gameMatrix[:10]), copy.copy(currentBlock)]
    modelInput = preProcessData([moveState])[0]
    result = model.predict(modelInput)[0]
    
    return np.argmax(result)
    
    
#play the game using the model's predictions and makeMove()
#Keep making moves in a loop. 
#If the total height of the tower was reduced, save the move for training
time.sleep(5) #Wait for 5 seconds so the user can start up the game
x_pad, y_pad, x_width, y_length = autoCalibrate()
for i in range(numberOfGames):
    clear_output()
    print("Progress: game " + str(i+1) + " out of " + str(numberOfGames))

    startGame()
    time.sleep(1) #This is so the game will have time to load
    giveUp = False #This will be used to end the loop
    lastMove = None #Data about the previous move
    lastMoveState = None #State of the game at prev. move
    lastHeight = -1 #Height of the prev. gamestate

    #Let's get the current block during the countdown
    im=pyscreenshot.grab(bbox =(x_pad,y_pad,x_width,y_length))
    im.save("game.png")

    frame = cv2.imread("game.png")
    frame1 = frame[100:172, 546:668]
    frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    frame2 = frame[172:244, 546:668]
    frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    frame3 = frame[244:309, 546:668]
    frame3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)

    currentBlock = getNextBlock(frame1)

    time.sleep(3) #Let's wait for 3 more seconds so the countdown menu ends

    while not giveUp:
        im=pyscreenshot.grab(bbox =(x_pad,y_pad,x_width,y_length))
        im.save("game.png")

        frame = cv2.imread("game.png")
        frame1 = frame[100:172, 546:668]
        frame1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        frame2 = frame[172:244, 546:668]
        frame2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        frame3 = frame[244:309, 546:668]
        frame3 = cv2.cvtColor(frame3, cv2.COLOR_BGR2GRAY)

        gameMatrix = getAllBlocks()
        h = getHeight(gameMatrix)
        nextBlock1 = getNextBlock(frame1)
        nextBlock2 = getNextBlock(frame2)
        nextBlock3 = getNextBlock(frame3)

        #If the tower is over 10 blocks high, the bot ends the game
        if(h > 10):
            giveUp = True

        move = predict(gameMatrix, currentBlock)

        if(not giveUp):
            makeMove(move, x_pad, y_pad)

        #If the last move reduced the height of the tower,
        #it was a good move and will be recorded
        if(h < lastHeight):
            completedLines += 1

        #Save the current data for future use
        lastHeight = h
        lastMoveState = [copy.deepcopy(gameMatrix[:10]), copy.copy(currentBlock)]
        lastMove = move
        currentBlock = nextBlock1
    #close the game and start a new one
    endGame(x_pad, y_pad)

print("The bot played " + str(numberOfGames) + " games and made " + str(completedLines) + " moves that completed a horizontal line.")

Progress: game 3 out of 3
The bot played 3 games and made 1 moves that completed a horizontal line.


In [ ]:
#Making tetris into excecutable
# File -> Download As -> Python
# conda install -c conda-forge pyinstaller
# conda install -c anaconda pywin32
# python TetrisBot_Main.py
# pyinstallef -F TetrisBot_main.py
# pyinstaller --onefile --add-data "red.png;." --add-data "green.png;." --add-data "yellow.png;." --add-data "orange.png;." --add-data "teal.png;." --add-data "purple.png;." --add-data "blue.png;." --add-data "hold.png;." --add-data "next.png;." --add-data "play.png;." --add-data "mudel.h5;." TetrisBot_main.py